In [ ]:
import os
import torch
import numpy as np

from ase.io import read
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io import Trajectory
from ase import units
from ase.io import read as aseread


import hippynn
from hippynn.experiment.serialization import load_model_from_cwd
from hippynn.interfaces.ase_interface import HippynnCalculator
from pathlib import Path

In [38]:
def calc_E_F(model_path, data_path, prefix, arrays=False, Z=None, R=None):

    # Load the model
    current_dir = os.getcwd() 
    os.chdir(model_path)
    model = load_model_from_cwd(map_location=torch.device('cpu')) # CPU runs for small molecules usually faster
    os.chdir(current_dir) 
    predictor = hippynn.graphs.Predictor.from_graph(model) 
    predictor.to(torch.float64)
    if arrays == False:
    # Load the molecule
        Z = np.load(data_path / f'{prefix}Z.npy')  # Atomic numbers
        R = np.load(data_path / f'{prefix}R.npy')  # Positions

        # Convert numpy arrays to torch tensors
        Z = torch.tensor(Z, dtype=torch.long)
        R = torch.tensor(R, dtype=torch.float64)

    else: 
        Z == torch.tensor(Z, dtype=torch.long)
        R == torch.tensor(R, dtype=torch.float64)
    # Forward pass to compute energy
    results = predictor(Z=Z, R=R)

    return results

In [7]:
pwd

'/vast/projects/ml4chem/NikitaFedik/GITHUB/ml_tps_si'

## Alanine Dipeptide

In [ ]:
results = calc_E_F(
    model_path=Path('models/hipnnts_data_ani1x_only_before_al_seed533257'),
    data_path=Path('data/AD/thermal_MD_10k/'),
    prefix='ad_'
    )

In [17]:
E = results['T'].reshape(results['T'].shape[0]) #get E 
F = results['forces'] # this is actually gradients! (- force)
F = -F # convert to atomic forces to match Gaussian output

In [18]:
E_shifted = E - torch.mean(E) # remove any arbitrary shifts to get relative formation E in kcal/mol

In [19]:
# check that values make sense
E_shifted

tensor([-7.0238,  6.4016,  9.9858,  ...,  0.0295,  5.6417,  3.3694],
       dtype=torch.float64)

In [20]:
np.save('data/AD/thermal_MD_10k/ad_E_hippnts_kcal_mol.npy', E_shifted.numpy())
np.save('data/AD/thermal_MD_10k/ad_forces_hippnts_kcal_mol_A.npy', F.numpy())

### Azobenzene

In [21]:
az_results = calc_E_F(
    model_path=Path('models/hipnnts_data_ani1x_only_before_al_seed533257'),
    data_path=Path('data/AZ/thermal_MD_10k/'),
    prefix='az_'
    )

In [22]:
E_az = az_results['T'].reshape(results['T'].shape[0]) #get E 
F_az = az_results['forces'] # this is actually gradients! (- force)
F_az = -F_az # convert to atomic forces to match Gaussian output
E_az_shifted = E_az - torch.mean(E_az) # remove any arbitrary shifts to get relative formation E in kcal/mol

np.save('data/AZ/thermal_MD_10k/az_E_hippynn_kcal_mol.npy', E_az_shifted.numpy())  
np.save('data/AZ/thermal_MD_10k/az_F_hippynn_kcal_mol_A.npy', F_az.numpy())


#### cs-inversion

In [39]:
cs_inv_xyz_path = Path('data/AZ/cs-inversion/AZ_cs-DFT_inversion_path.xyz')
cs_inv_atoms_list = aseread(cs_inv_xyz_path, index=":")

cs_inv_Z = torch.tensor([atoms.get_atomic_numbers() for atoms in cs_inv_atoms_list], dtype=torch.long)
cs_inv_R = torch.stack([torch.tensor(atoms.get_positions(), dtype=torch.float64) for atoms in cs_inv_atoms_list])

cs_inv_results = calc_E_F(('models/hipnnts_data_ani1x_only_before_al_seed533257'),  
                          data_path = None,
                          arrays=True,
                          Z=cs_inv_Z, 
                          R=cs_inv_R, 
                          prefix='cs_inv_',)

/vast/home/fns/.conda/envs/qm/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/vast/home/fns/.conda/envs/qm/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [48]:
cs_inv_E = cs_inv_results['T'].reshape(cs_inv_results['T'].shape[0], 1)
cs_inv_F = cs_inv_results['forces'] # this is actually gradients! (- force)
cs_inv_F = -cs_inv_F # convert to atomic forces to match Gaussian output
cs_inv_E_shifted = cs_inv_E - torch.mean(cs_inv_E) # remove any arbitrary shifts to get relative formation E in kcal/mol
np.save('data/AZ/cs-inversion/cs_inv_E_hippynn_kcal_mol.npy', cs_inv_E_shifted.numpy())
# np.save('data/AZ/cs-inversion/cs_inv_F_hipnnts_kcal_mol_A.npy', cs_inv_F.numpy())